# Lesson 03 - Under and Over Fitting

The previous model which we have used transfer learning for Cats and Dogs Redux Kaggle competition was under fitting. The training accuracy was lower than the validation accuracy. This is due to dropout operations in dense (fully connected) layers in VGG-16 model. We should somehow remove the dropout operations.

In [1]:
# import the model and utils
import numpy as np
from utils import *
from importlib import reload
import vgg16_for_keras2
reload(vgg16_for_keras2)
from vgg16_for_keras2 import Vgg16

Using TensorFlow backend.


In [2]:
vgg = Vgg16()

In [3]:
path = './data/sample-fast-ai/'
batch_size = 32

In [4]:
%ls data/sample-fast-ai/

train/  valid/


In [5]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)

Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [6]:
# change the output layer from 1000 nodes to 2 nodes
vgg.finetune(batches)

In [7]:
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [9]:
# get layers
layers = vgg.model.layers
len(layers)

38

In [36]:
from keras.layers import Convolution2D
from keras.models import Sequential

In [32]:
# lets check the layer types
print(type(layers[37]))
print(type(layers[36]))
print(type(layers[35]))
print(type(layers[23]))
print(type(layers[23]) is Convolution2D)

<class 'keras.layers.core.Dense'>
<class 'keras.layers.core.Dropout'>
<class 'keras.layers.core.Dense'>
<class 'keras.layers.convolutional.Conv2D'>
True


In [39]:
# find the index of last convolutional layer
last_conv_index = [index for index,layer in enumerate(layers)
                      if type(layer) is Convolution2D][-1]
print(last_conv_index)

30


In [40]:
print(type(layers[30]))
print(type(layers[31]))
print(type(layers[32]))

<class 'keras.layers.convolutional.Conv2D'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.core.Flatten'>


In [37]:
conv_layers = layers[:last_conv_index+1]
conv_model = Sequential(conv_layers)

In [41]:
val_features = conv_model.predict_generator(val_batches, val_batches.samples//batch_size)

In [42]:
val_features.shape

(192, 512, 14, 14)

In [43]:
train_features = conv_model.predict_generator(batches, batches.samples//batch_size)

In [44]:
train_features.shape

(192, 512, 14, 14)

In [46]:
type(train_features)

numpy.ndarray

In [48]:
import bcolz

In [58]:
# save bottleneck fetures for later use
features_dir ='bottleneck_features'

%mkdir -p features_dir
c = bcolz.carray(val_features, rootdir=features_dir+'/val_features', mode='w')
c.flush()

c = bcolz.carray(train_features, rootdir=features_dir+'/train_features', mode='w')
c.flush()

%ls bottleneck_features

train_features/  train_features.bc/  val_features/  val_features.bc/


In [59]:
train_feattures = bcolz.open(features_dir+'/train_features')
val_features = bcolz.open(features_dir+'/val_features')

In [60]:
print(train_feattures.shape)
print(val_features.shape)

(192, 512, 14, 14)
(192, 512, 14, 14)


In [61]:
from keras.layers import MaxPooling2D, Dense, Flatten, Dropout

In [65]:
conv_layers[-1].output_shape

(None, 512, 14, 14)

In [68]:
# create fully connected model
fc_model = Sequential([
    MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.),
    Dense(4096, activation='relu'),
    Dropout(0.),
    Dense(2, activation='softmax'),
]
)